In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize

In [2]:
import gc
gc.enable()

In [8]:
def train_model(train_, val_x_, val_y_, test_, y_, folds_):
    feats_ = [f_ for f_ in test_.columns if f_ not in ['SK_ID_CURR']]
    oof_preds = np.zeros(train_.shape[0])
    val_preds = np.zeros(val_x_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    

    
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(train_)):
        #print(train_.type)
        trn_x, trn_y = pd.DataFrame(train_).iloc[trn_idx], pd.DataFrame(y_).iloc[trn_idx]
        val_x, val_y = pd.DataFrame(train_).iloc[val_idx], pd.DataFrame(y_).iloc[val_idx]
        
        clf = LogisticRegression(
            penalty = 'l1',
            C = 10.0,
            class_weight = 'balanced',
            random_state = 14,
            n_jobs = 72
            )
        clf.fit(trn_x, trn_y)
        oof_preds[val_idx] = clf.predict_proba(val_x)[:, 1]
        val_preds += clf.predict_proba(pd.DataFrame(val_x_[feats_]))[:, 1] / folds_.n_splits
        sub_preds += clf.predict_proba(pd.DataFrame(test_[feats_]))[:, 1] / folds_.n_splits
        print('fold %2d validate AUC score %.6f'%(n_fold + 1,roc_auc_score(val_y_, val_preds* folds_.n_splits) ))
        print('fold %2d AUC %.6f'%(n_fold+1, roc_auc_score(val_y, oof_preds[val_idx])))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
    print('validate AUC score %.6f'%roc_auc_score(val_y_, val_preds))
    print('full AUC score %.6f'%roc_auc_score(y_, oof_preds))
    test_['TARGET'] = sub_preds
    
    return oof_preds, test_[['SK_ID_CURR', 'TARGET']]

In [9]:
train = pd.read_csv('../data/train_.csv')
test = pd.read_csv('../data/test_.csv')
y = pd.read_csv('../data/y_.csv')
val_x = pd.read_csv('../data/val_x_.csv')
val_y = pd.read_csv('../data/val_y_.csv')
folds = KFold(n_splits=5, shuffle=True, random_state=0)
features = val_x.columns
feats = [f for f in features if f not in ['SK_ID_CURR']]

In [10]:
train = normalize(train)
test = normalize(test)
val_x = normalize(val_x)

In [11]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)
val_x = pd.DataFrame(val_x)
train.columns = feats
test.columns = features
val_x.columns = features

In [12]:
oof_preds, test_preds = train_model(train, val_x, val_y['TARGET'].values.ravel(), test, y['0'].values.ravel(), folds)
test_preds.to_csv('../data/lr_submission.csv', index=False)

F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\KG\software\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 72.
  " = {}.".format(self.n_jobs))


fold  1 validate AUC score 0.568079
fold  1 AUC 0.950656
fold  2 validate AUC score 0.567908
fold  2 AUC 0.951421
fold  3 validate AUC score 0.568002
fold  3 AUC 0.951863
fold  4 validate AUC score 0.567944
fold  4 AUC 0.949995
fold  5 validate AUC score 0.567984
fold  5 AUC 0.950631
validate AUC score 0.567984
full AUC score 0.950915
